# Users

I hate the word "users". However, calling them "people" would be a form of
discrimination against the machines that could be using this product as well.
And we don't want that.

In [ ]:
//| export

import { getProfile } from "tinychat/bsky.ts";
import { Database } from "tinychat/db.ts";

export const syncUser = async ({ did, db }: { did: string; db: Database }) => {
  const profile = await getProfile(did);
  db.prepare(
    `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :displayName, :avatar, :description
        ) ON CONFLICT(did) DO UPDATE SET
          handle = COALESCE(:handle, handle),
          display_name = COALESCE(:displayName, display_name),
          avatar = COALESCE(:avatar, avatar),
          description = COALESCE(:description, description
        )`,
  ).run({
    did,
    handle: profile.handle,
    displayName: profile.displayName || null,
    avatar: profile.avatar || null,
    description: profile.description || null,
  });
};

# Test helpers

In [ ]:
//| export

import { Database } from "tinychat/db.ts";

export const createTestUser = (
  { db, user }: { db: Database; user: { did: string; handle: string } },
) => {
  // this test user does not have some common fields like display name and avatar not set intentionally
  // to mimic bare bones setup - handy for testing record validations with missing fields etc.
  db.prepare(
    "INSERT INTO users (did, handle) VALUES (:did, :handle)",
  ).run(user);
};

export const createDefaultTestUser = ({ db }: { db: Database }) => {
  // this test user does not have some common fields like display name and avatar not set intentionally
  // to mimic bare bones setup - handy for testing record validations with missing fields etc.
  createTestUser({
    db,
    user: {
      did: "did:plc:ubdeopbbkbgedccgbum7dhsh",
      handle: "callmephilip.com",
    },
  });
};